In [1]:

# Campus Navigation and Utility Planner


# Building Node for Trees 
class Building:
    def __init__(self, building_id, name, location):
        self.building_id = building_id
        self.name = name
        self.location = location
        self.left = None
        self.right = None
        self.height = 1  # for AVL

    def __str__(self):
        return f"{self.building_id}: {self.name} ({self.location})"


# Binary Search Tree Implementation

class BST:
    def __init__(self):
        self.root = None

    def insert(self, root, building):
        if root is None:
            return building
        if building.building_id < root.building_id:
            root.left = self.insert(root.left, building)
        else:
            root.right = self.insert(root.right, building)
        return root

    def inorder(self, root):
        return [] if root is None else self.inorder(root.left) + [root] + self.inorder(root.right)

    def preorder(self, root):
        return [] if root is None else [root] + self.preorder(root.left) + self.preorder(root.right)

    def postorder(self, root):
        return [] if root is None else self.postorder(root.left) + self.postorder(root.right) + [root]

    def search(self, root, building_id):
        if root is None or root.building_id == building_id:
            return root
        if building_id < root.building_id:
            return self.search(root.left, building_id)
        return self.search(root.right, building_id)


# AVL Tree Implementation

class AVLTree:
    def __init__(self):
        self.root = None

    def get_height(self, root):
        return 0 if not root else root.height

    def get_balance(self, root):
        return 0 if not root else self.get_height(root.left) - self.get_height(root.right)

    def rotate_right(self, y):
        x = y.left
        T2 = x.right
        x.right = y
        y.left = T2
        y.height = 1 + max(self.get_height(y.left), self.get_height(y.right))
        x.height = 1 + max(self.get_height(x.left), self.get_height(x.right))
        return x

    def rotate_left(self, x):
        y = x.right
        T2 = y.left
        y.left = x
        x.right = T2
        x.height = 1 + max(self.get_height(x.left), self.get_height(x.right))
        y.height = 1 + max(self.get_height(y.left), self.get_height(y.right))
        return y

    def insert(self, root, building):
        if root is None:
            return building
        if building.building_id < root.building_id:
            root.left = self.insert(root.left, building)
        else:
            root.right = self.insert(root.right, building)

        root.height = 1 + max(self.get_height(root.left), self.get_height(root.right))
        balance = self.get_balance(root)

        # LL
        if balance > 1 and building.building_id < root.left.building_id:
            print("LL Rotation on", root.name)
            return self.rotate_right(root)
        # RR
        if balance < -1 and building.building_id > root.right.building_id:
            print("RR Rotation on", root.name)
            return self.rotate_left(root)
        # LR
        if balance > 1 and building.building_id > root.left.building_id:
            print("LR Rotation on", root.name)
            root.left = self.rotate_left(root.left)
            return self.rotate_right(root)
        # RL
        if balance < -1 and building.building_id < root.right.building_id:
            print("RL Rotation on", root.name)
            root.right = self.rotate_right(root.right)
            return self.rotate_left(root)
        return root

    def inorder(self, root):
        return [] if root is None else self.inorder(root.left) + [root] + self.inorder(root.right)


# Graph Implementation

from collections import defaultdict, deque

class Graph:
    def __init__(self):
        self.graph = defaultdict(list)
        self.weights = {}

    def add_edge(self, u, v, w):
        self.graph[u].append(v)
        self.graph[v].append(u)
        self.weights[(u, v)] = w
        self.weights[(v, u)] = w

    def bfs(self, start):
        visited, queue, result = set(), deque([start]), []
        while queue:
            node = queue.popleft()
            if node not in visited:
                visited.add(node)
                result.append(node)
                queue.extend(self.graph[node])
        return result

    def dfs(self, start, visited=None):
        if visited is None:
            visited = set()
        visited.add(start)
        for nbr in self.graph[start]:
            if nbr not in visited:
                self.dfs(nbr, visited)
        return visited

    def dijkstra(self, start):
        import heapq
        distances = {v: float('inf') for v in self.graph}
        distances[start] = 0
        pq = [(0, start)]
        while pq:
            current_dist, current_node = heapq.heappop(pq)
            for neighbor in self.graph[current_node]:
                distance = current_dist + self.weights[(current_node, neighbor)]
                if distance < distances[neighbor]:
                    distances[neighbor] = distance
                    heapq.heappush(pq, (distance, neighbor))
        return distances

    def kruskal(self):
        parent, rank = {}, {}
        def find(v):
            if parent[v] != v:
                parent[v] = find(parent[v])
            return parent[v]

        def union(v1, v2):
            root1, root2 = find(v1), find(v2)
            if root1 != root2:
                if rank[root1] < rank[root2]:
                    parent[root1] = root2
                else:
                    parent[root2] = root1
                    if rank[root1] == rank[root2]:
                        rank[root1] += 1

        for v in self.graph:
            parent[v] = v
            rank[v] = 0

        edges = sorted(self.weights.items(), key=lambda x: x[1])
        mst = []
        for (u, v), w in edges:
            if find(u) != find(v):
                union(u, v)
                mst.append((u, v, w))
        return mst


# Expression Tree for Energy Bill Calculation

class ExpressionTree:
    class Node:
        def __init__(self, value):
            self.value = value
            self.left = None
            self.right = None

    def build(self, postfix):
        stack = []
        for token in postfix:
            if token not in "+-*/":
                stack.append(self.Node(token))
            else:
                node = self.Node(token)
                node.right = stack.pop()
                node.left = stack.pop()
                stack.append(node)
        return stack.pop()

    def evaluate(self, root):
        if root is None:
            return 0
        if root.value not in "+-*/":
            return float(root.value)
        left = self.evaluate(root.left)
        right = self.evaluate(root.right)
        if root.value == '+': return left + right
        if root.value == '-': return left - right
        if root.value == '*': return left * right
        if root.value == '/': return left / right



# Example Run

if __name__ == "__main__":
    # Create BST and AVL Trees
    buildings = [
        (10, "Library", "Central Block"),
        (20, "CSE Dept", "North Wing"),
        (5, "Admin", "Main Entrance"),
        (15, "Hostel", "South Block")
    ]

    print("\n=== Binary Search Tree ===")
    bst = BST()
    for b in buildings:
        bst.root = bst.insert(bst.root, Building(*b))
    print("Inorder Traversal (BST):", [b.name for b in bst.inorder(bst.root)])

    print("\n=== AVL Tree (with Rotations) ===")
    avl = AVLTree()
    for b in buildings:
        avl.root = avl.insert(avl.root, Building(*b))
    print("Inorder Traversal (AVL):", [b.name for b in avl.inorder(avl.root)])

    # Graph Representation
    print("\n=== Campus Graph ===")
    g = Graph()
    g.add_edge("Library", "Admin", 4)
    g.add_edge("Library", "CSE Dept", 2)
    g.add_edge("CSE Dept", "Hostel", 5)
    g.add_edge("Admin", "Hostel", 10)

    print("BFS from Library:", g.bfs("Library"))
    print("DFS from Library:", list(g.dfs("Library")))
    print("Shortest Paths (Dijkstra):", g.dijkstra("Library"))
    print("Minimum Spanning Tree (Kruskal):", g.kruskal())

    # Expression Tree Example
    print("\n=== Energy Bill Calculation ===")
    exp = ExpressionTree()
    postfix = ["10", "20", "+", "3", "*"]  # (10 + 20) * 3
    root = exp.build(postfix)
    print("Result:", exp.evaluate(root))



=== Binary Search Tree ===
Inorder Traversal (BST): ['Admin', 'Library', 'Hostel', 'CSE Dept']

=== AVL Tree (with Rotations) ===
Inorder Traversal (AVL): ['Admin', 'Library', 'Hostel', 'CSE Dept']

=== Campus Graph ===
BFS from Library: ['Library', 'Admin', 'CSE Dept', 'Hostel']
DFS from Library: ['Admin', 'Hostel', 'CSE Dept', 'Library']
Shortest Paths (Dijkstra): {'Library': 0, 'Admin': 4, 'CSE Dept': 2, 'Hostel': 7}
Minimum Spanning Tree (Kruskal): [('Library', 'CSE Dept', 2), ('Library', 'Admin', 4), ('CSE Dept', 'Hostel', 5)]

=== Energy Bill Calculation ===
Result: 90.0
